In [8]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi! "}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi! '), AIMessage(content='How are you?')]}

In [9]:
memory.save_context({"input": "Hi! "}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?')]}

In [10]:
memory.save_context({"input": "Hi! "}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?')]}

In [11]:
memory.save_context({"input": "Hi! "}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?')]}

In [12]:
memory.save_context({"input": "Hi! "}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?')]}

In [13]:
memory.save_context({"input": "Hi! "}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi! '),
  AIMessage(content='How are you?')]}